In [1]:
#import libraries
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [2]:
#load the transcript data for S&P 100 for 2020 only
#read csv that includes stock data
transcribed_list_org= pd.read_csv('transcribed_transcript_list2.csv')
transcribed_list_org.head(10)


,Unnamed: 0,date,ticker,title,transcript,transcript_prep_remarks,url
0,0,10/29/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q4 2020 Earnings Conf...,"['Operator', 'Good day everyone and welcome to...",/article/4382943-apple-inc-aapl-ceo-tim-cook-o...
1,1,7/30/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q3 2020 Results Confe...,"['Operator', ""Good day, everyone. Welcome to t...",/article/4362707-apple-inc-aapl-ceo-tim-cook-o...
2,2,4/30/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q2 2020 Results Confe...,"['Operator', 'Good day everyone. Welcome to th...",/article/4341792-apple-inc-aapl-ceo-tim-cook-o...
3,3,1/28/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q1 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q1 2020 Results Confe...,"['Operator', ""Good day, everyone. Welcome to t...",/article/4319666-apple-inc-aapl-ceo-tim-cook-o...
4,8,10/30/2020,ABBV,AbbVie Inc.'s (ABBV) CEO Rick Gonzalez on Q3 2...,AbbVie Inc. (NYSE:ABBV) Q3 2020 Results Earnin...,"['Operator', 'Good morning and thank you for s...",/article/4383381-abbvie-inc-s-abbv-ceo-rick-go...
5,10,7/31/2020,ABBV,AbbVie Inc. (ABBV) CEO Rick Gonzalez on Q2 202...,AbbVie Inc. (NYSE:ABBV) Q2 2020 Earnings Confe...,"['Operator', 'Good morning. And thank you for ...",/article/4363134-abbvie-inc-abbv-ceo-rick-gonz...
6,11,5/1/2020,ABBV,AbbVie Inc. (ABBV) CEO Rick Gonzalez on Q1 202...,AbbVie Inc. (NYSE:ABBV) Q1 2020 Earnings Confe...,"['Operator', 'Good morning and thank you for s...",/article/4342124-abbvie-inc-abbv-ceo-rick-gonz...
7,12,2/7/2020,ABBV,AbbVie Inc. (ABBV) CEO Rick Gonzalez on Q4 201...,AbbVie Inc. (NYSE:ABBV) Q4 2019 Earnings Confe...,"['Operator', 'Good morning and thank you for s...",/article/4322499-abbvie-inc-abbv-ceo-rick-gonz...
8,14,10/21/2020,ABT,Abbott Laboratories (ABT) CEO Robert Ford on Q...,Abbott Laboratories (NYSE:ABT) Q3 2020 Earning...,"['Operator', ""Good morning and thank you for s...",/article/4380361-abbott-laboratories-abt-ceo-r...
9,16,7/16/2020,ABT,"Abbott Laboratories (ABT) CEO, Robert Ford on ...",Abbott Laboratories (NYSE:ABT) Q2 2020 Earning...,"['Operator', 'Good morning and thank you for s...",/article/4358872-abbott-laboratories-abt-ceo-r...


In [3]:
#store the data
#transcript_prep_remarks
transcribed_list_org.head(5)

,Unnamed: 0,date,ticker,title,transcript,transcript_prep_remarks,url
0,0,10/29/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q4 2020 Earnings Conf...,"['Operator', 'Good day everyone and welcome to...",/article/4382943-apple-inc-aapl-ceo-tim-cook-o...
1,1,7/30/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q3 2020 Results Confe...,"['Operator', ""Good day, everyone. Welcome to t...",/article/4362707-apple-inc-aapl-ceo-tim-cook-o...
2,2,4/30/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q2 2020 Results Confe...,"['Operator', 'Good day everyone. Welcome to th...",/article/4341792-apple-inc-aapl-ceo-tim-cook-o...
3,3,1/28/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q1 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q1 2020 Results Confe...,"['Operator', ""Good day, everyone. Welcome to t...",/article/4319666-apple-inc-aapl-ceo-tim-cook-o...
4,8,10/30/2020,ABBV,AbbVie Inc.'s (ABBV) CEO Rick Gonzalez on Q3 2...,AbbVie Inc. (NYSE:ABBV) Q3 2020 Results Earnin...,"['Operator', 'Good morning and thank you for s...",/article/4383381-abbvie-inc-s-abbv-ceo-rick-go...


In [4]:
transcribed_list_org.dtypes

Unnamed: 0                  int64
date                       object
ticker                     object
title                      object
transcript                 object
transcript_prep_remarks    object
url                        object
dtype: object

In [5]:
transcribed_list_org['date_rev2']= pd.to_datetime(transcribed_list_org['date'])
transcribed_list_org['date_rev3']= transcribed_list_org['date_rev2'].dt.strftime('%m/%d/%Y')

In [6]:
transcribed_list_org.head(5)

,Unnamed: 0,date,ticker,title,transcript,transcript_prep_remarks,url,date_rev2,date_rev3
0,0,10/29/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q4 2020 Earnings Conf...,"['Operator', 'Good day everyone and welcome to...",/article/4382943-apple-inc-aapl-ceo-tim-cook-o...,2020-10-29,10/29/2020
1,1,7/30/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q3 2020 Results Confe...,"['Operator', ""Good day, everyone. Welcome to t...",/article/4362707-apple-inc-aapl-ceo-tim-cook-o...,2020-07-30,07/30/2020
2,2,4/30/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q2 2020 Results Confe...,"['Operator', 'Good day everyone. Welcome to th...",/article/4341792-apple-inc-aapl-ceo-tim-cook-o...,2020-04-30,04/30/2020
3,3,1/28/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q1 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q1 2020 Results Confe...,"['Operator', ""Good day, everyone. Welcome to t...",/article/4319666-apple-inc-aapl-ceo-tim-cook-o...,2020-01-28,01/28/2020
4,8,10/30/2020,ABBV,AbbVie Inc.'s (ABBV) CEO Rick Gonzalez on Q3 2...,AbbVie Inc. (NYSE:ABBV) Q3 2020 Results Earnin...,"['Operator', 'Good morning and thank you for s...",/article/4383381-abbvie-inc-s-abbv-ceo-rick-go...,2020-10-30,10/30/2020


In [7]:
transcribed_list_org= transcribed_list_org.rename(columns={'ticker': 'Symbol'})
transcribed_list_org.head(3)

,Unnamed: 0,date,Symbol,title,transcript,transcript_prep_remarks,url,date_rev2,date_rev3
0,0,10/29/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q4 2020 Earnings Conf...,"['Operator', 'Good day everyone and welcome to...",/article/4382943-apple-inc-aapl-ceo-tim-cook-o...,2020-10-29,10/29/2020
1,1,7/30/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q3 2020 Results Confe...,"['Operator', ""Good day, everyone. Welcome to t...",/article/4362707-apple-inc-aapl-ceo-tim-cook-o...,2020-07-30,07/30/2020
2,2,4/30/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q2 2020 Results Confe...,"['Operator', 'Good day everyone. Welcome to th...",/article/4341792-apple-inc-aapl-ceo-tim-cook-o...,2020-04-30,04/30/2020


In [8]:
#display the data


In [9]:
#load the price data for S&P 100 for 2020 only

price_list= pd.read_csv('combined_snp100_data.csv')
price_list.head(10)

,Unnamed: 0,Symbol,Name,Date,Adj_Close,Close,High,Low,Open,Volume
0,0,AAPL,Apple Inc.,2020-01-02,74.444603,75.087502,75.150002,73.797501,74.059998,"135,480,400"
1,1,AAPL,Apple Inc.,2020-01-03,73.720840,74.357498,75.144997,74.125000,74.287498,"146,322,800"
2,2,AAPL,Apple Inc.,2020-01-06,74.308266,74.949997,74.989998,73.187500,73.447502,"118,387,200"
3,3,AAPL,Apple Inc.,2020-01-07,73.958794,74.597504,75.224998,74.370003,74.959999,"108,872,000"
4,4,AAPL,Apple Inc.,2020-01-08,75.148521,75.797501,76.110001,74.290001,74.290001,"132,079,200"
5,5,AAPL,Apple Inc.,2020-01-09,76.744728,77.407501,77.607498,76.550003,76.809998,"170,108,400"
6,6,AAPL,Apple Inc.,2020-01-10,76.918221,77.582497,78.167503,77.062500,77.650002,"140,644,800"
7,7,AAPL,Apple Inc.,2020-01-13,78.561531,79.239998,79.267502,77.787498,77.910004,"121,532,000"
8,8,AAPL,Apple Inc.,2020-01-14,77.500702,78.169998,79.392502,78.042503,79.175003,"161,954,400"
9,9,AAPL,Apple Inc.,2020-01-15,77.168564,77.834999,78.875000,77.387497,77.962502,"121,923,600"


In [10]:
#price_list=price_list.drop(['Unnamed: 0'],1)
price_list.head(10)


,Unnamed: 0,Symbol,Name,Date,Adj_Close,Close,High,Low,Open,Volume
0,0,AAPL,Apple Inc.,2020-01-02,74.444603,75.087502,75.150002,73.797501,74.059998,"135,480,400"
1,1,AAPL,Apple Inc.,2020-01-03,73.720840,74.357498,75.144997,74.125000,74.287498,"146,322,800"
2,2,AAPL,Apple Inc.,2020-01-06,74.308266,74.949997,74.989998,73.187500,73.447502,"118,387,200"
3,3,AAPL,Apple Inc.,2020-01-07,73.958794,74.597504,75.224998,74.370003,74.959999,"108,872,000"
4,4,AAPL,Apple Inc.,2020-01-08,75.148521,75.797501,76.110001,74.290001,74.290001,"132,079,200"
5,5,AAPL,Apple Inc.,2020-01-09,76.744728,77.407501,77.607498,76.550003,76.809998,"170,108,400"
6,6,AAPL,Apple Inc.,2020-01-10,76.918221,77.582497,78.167503,77.062500,77.650002,"140,644,800"
7,7,AAPL,Apple Inc.,2020-01-13,78.561531,79.239998,79.267502,77.787498,77.910004,"121,532,000"
8,8,AAPL,Apple Inc.,2020-01-14,77.500702,78.169998,79.392502,78.042503,79.175003,"161,954,400"
9,9,AAPL,Apple Inc.,2020-01-15,77.168564,77.834999,78.875000,77.387497,77.962502,"121,923,600"


In [11]:
price_list.dtypes

Unnamed: 0      int64
Symbol         object
Name           object
Date           object
Adj_Close     float64
Close         float64
High          float64
Low           float64
Open          float64
Volume         object
dtype: object

In [12]:
price_list['date_rev2']= pd.to_datetime(price_list['Date'])
price_list['date_rev3']= price_list['date_rev2'].dt.strftime('%m/%d/%Y')

In [13]:
price_list.head(5)

,Unnamed: 0,Symbol,Name,Date,Adj_Close,Close,High,Low,Open,Volume,date_rev2,date_rev3
0,0,AAPL,Apple Inc.,2020-01-02,74.444603,75.087502,75.150002,73.797501,74.059998,"135,480,400",2020-01-02,01/02/2020
1,1,AAPL,Apple Inc.,2020-01-03,73.720840,74.357498,75.144997,74.125000,74.287498,"146,322,800",2020-01-03,01/03/2020
2,2,AAPL,Apple Inc.,2020-01-06,74.308266,74.949997,74.989998,73.187500,73.447502,"118,387,200",2020-01-06,01/06/2020
3,3,AAPL,Apple Inc.,2020-01-07,73.958794,74.597504,75.224998,74.370003,74.959999,"108,872,000",2020-01-07,01/07/2020
4,4,AAPL,Apple Inc.,2020-01-08,75.148521,75.797501,76.110001,74.290001,74.290001,"132,079,200",2020-01-08,01/08/2020


In [14]:
merge_price_trans=pd.merge(transcribed_list_org, price_list, on=['Symbol', 'date_rev3'])
merge_price_trans.head(3)
merge_price_trans.columns


Index(['Unnamed: 0_x', 'date', 'Symbol', 'title', 'transcript',
       'transcript_prep_remarks', 'url', 'date_rev2_x', 'date_rev3',
       'Unnamed: 0_y', 'Name', 'Date', 'Adj_Close', 'Close', 'High', 'Low',
       'Open', 'Volume', 'date_rev2_y'],
      dtype='object')

In [15]:
merge_price_trans.head(3)

,Unnamed: 0_x,date,Symbol,title,transcript,transcript_prep_remarks,url,date_rev2_x,date_rev3,Unnamed: 0_y,Name,Date,Adj_Close,Close,High,Low,Open,Volume,date_rev2_y
0,0,10/29/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q4 2020 Earnings Conf...,"['Operator', 'Good day everyone and welcome to...",/article/4382943-apple-inc-aapl-ceo-tim-cook-o...,2020-10-29,10/29/2020,209,Apple Inc.,2020-10-29,115.121384,115.320000,116.930000,112.199997,112.370003,"146,129,200",2020-10-29
1,1,7/30/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q3 2020 Results Confe...,"['Operator', ""Good day, everyone. Welcome to t...",/article/4362707-apple-inc-aapl-ceo-tim-cook-o...,2020-07-30,07/30/2020,145,Apple Inc.,2020-07-30,95.851517,96.190002,96.297501,93.767502,94.187500,"158,130,000",2020-07-30
2,2,4/30/2020,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q2 2020 Results Confe...,"['Operator', 'Good day everyone. Welcome to th...",/article/4341792-apple-inc-aapl-ceo-tim-cook-o...,2020-04-30,04/30/2020,82,Apple Inc.,2020-04-30,72.993935,73.449997,73.632500,72.087502,72.489998,"183,064,000",2020-04-30


In [16]:
merge_price_trans2= merge_price_trans.drop(['Unnamed: 0_x','date', 'date_rev2_x','Date', 'date_rev2_y' ],1)
merge_price_trans2.head(3)
print(merge_price_trans2.columns)

Index(['Symbol', 'title', 'transcript', 'transcript_prep_remarks', 'url',
       'date_rev3', 'Unnamed: 0_y', 'Name', 'Adj_Close', 'Close', 'High',
       'Low', 'Open', 'Volume'],
      dtype='object')


In [17]:
merge_price_trans2= merge_price_trans2.rename(columns={'date_rev3':'Date'})
merge_price_trans2.head(3)

,Symbol,title,transcript,transcript_prep_remarks,url,Date,Unnamed: 0_y,Name,Adj_Close,Close,High,Low,Open,Volume
0,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q4 2020 Earnings Conf...,"['Operator', 'Good day everyone and welcome to...",/article/4382943-apple-inc-aapl-ceo-tim-cook-o...,10/29/2020,209,Apple Inc.,115.121384,115.320000,116.930000,112.199997,112.370003,"146,129,200"
1,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q3 2020 Results Confe...,"['Operator', ""Good day, everyone. Welcome to t...",/article/4362707-apple-inc-aapl-ceo-tim-cook-o...,07/30/2020,145,Apple Inc.,95.851517,96.190002,96.297501,93.767502,94.187500,"158,130,000"
2,AAPL,Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q2 2020 Results Confe...,"['Operator', 'Good day everyone. Welcome to th...",/article/4341792-apple-inc-aapl-ceo-tim-cook-o...,04/30/2020,82,Apple Inc.,72.993935,73.449997,73.632500,72.087502,72.489998,"183,064,000"


In [18]:
merge_price_trans2=merge_price_trans2[['Date', 'Symbol','Name','title', 'transcript', 'transcript_prep_remarks', 'url','Adj_Close', 'Close', 'High', 'Low', 'Open',
       'Volume']]
merge_price_trans2.head(3)


,Date,Symbol,Name,title,transcript,transcript_prep_remarks,url,Adj_Close,Close,High,Low,Open,Volume
0,10/29/2020,AAPL,Apple Inc.,Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q4 2020 Earnings Conf...,"['Operator', 'Good day everyone and welcome to...",/article/4382943-apple-inc-aapl-ceo-tim-cook-o...,115.121384,115.320000,116.930000,112.199997,112.370003,"146,129,200"
1,07/30/2020,AAPL,Apple Inc.,Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q3 2020 Results Confe...,"['Operator', ""Good day, everyone. Welcome to t...",/article/4362707-apple-inc-aapl-ceo-tim-cook-o...,95.851517,96.190002,96.297501,93.767502,94.187500,"158,130,000"
2,04/30/2020,AAPL,Apple Inc.,Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q2 2020 Results Confe...,"['Operator', 'Good day everyone. Welcome to th...",/article/4341792-apple-inc-aapl-ceo-tim-cook-o...,72.993935,73.449997,73.632500,72.087502,72.489998,"183,064,000"


In [5]:
#clean the data



In [19]:
#create a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

In [20]:
#create function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity


In [21]:
#create new columns for subjectivity and polarity on MERGED DATA

merge_price_trans2['Subjectivity']= merge_price_trans2['transcript_prep_remarks'].apply(getSubjectivity)

merge_price_trans2['Polarity']= merge_price_trans2['transcript_prep_remarks'].apply(getPolarity)

In [22]:
merge_price_trans2['Subjectivity'][0]

0.49211034059539516

In [23]:
#create function to get sentiment score using Sentiment Intensity Analyzer

def getSIA(text):
    sia=SentimentIntensityAnalyzer()
    
    sentiment=sia.polarity_scores(text)
    return sentiment

In [24]:
compound=[]
neg= []
pos = []
neu = []
SIA = 0

for i in range(0, len(merge_price_trans2['transcript_prep_remarks'])):
    SIA = getSIA(merge_price_trans2['transcript_prep_remarks'][i])
    compound.append(SIA['compound'])
    neg.append(SIA['neg'])
    neu.append(SIA['neu'])
    pos.append(SIA['pos'])

In [30]:
pos[:3]

[0.167, 0.171, 0.157]

In [31]:
SIA

{'neg': 0.029, 'neu': 0.817, 'pos': 0.154, 'compound': 1.0}

In [32]:
#store sentiment scores in dataframe
merge_price_trans2['Compound']=compound
merge_price_trans2['Negative']=neg
merge_price_trans2['Positive']=pos
merge_price_trans2['Neutral']=neu

In [33]:
merge_price_trans2.head(3)

,,Name,title,transcript,transcript_prep_remarks,url,Adj_Close,Close,High,Low,Open,Volume,Subjectivity,Polarity,Compound,Negative,Positive,Neutral
Date,Symbol,,,,,,,,,,,,,,,,,
10/29/2020,AAPL,Apple Inc.,Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q4 2020 Earnings Conf...,"['Operator', 'Good day everyone and welcome to...",/article/4382943-apple-inc-aapl-ceo-tim-cook-o...,115.121384,115.320000,116.930000,112.199997,112.370003,"146,129,200",0.492110,0.168114,1.0,0.036,0.167,0.797
07/30/2020,AAPL,Apple Inc.,Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q3 2020 Results Confe...,"['Operator', ""Good day, everyone. Welcome to t...",/article/4362707-apple-inc-aapl-ceo-tim-cook-o...,95.851517,96.190002,96.297501,93.767502,94.187500,"158,130,000",0.482856,0.164288,1.0,0.028,0.171,0.801
04/30/2020,AAPL,Apple Inc.,Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q2 2020 Results Confe...,"['Operator', 'Good day everyone. Welcome to th...",/article/4341792-apple-inc-aapl-ceo-tim-cook-o...,72.993935,73.449997,73.632500,72.087502,72.489998,"183,064,000",0.448222,0.193248,1.0,0.030,0.157,0.813


In [29]:
merge_price_trans2.set_index(['Date', 'Symbol'], inplace =True)
merge_price_trans2


,,Name,title,transcript,transcript_prep_remarks,url,Adj_Close,Close,High,Low,Open,Volume,Subjectivity,Polarity,Compound,Negative,Positive,Neutral
Date,Symbol,,,,,,,,,,,,,,,,,
10/29/2020,AAPL,Apple Inc.,Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q4 2020 Earnings Conf...,"['Operator', 'Good day everyone and welcome to...",/article/4382943-apple-inc-aapl-ceo-tim-cook-o...,115.121384,115.320000,116.930000,112.199997,112.370003,"146,129,200",0.492110,0.168114,1.0000,0.036,0.167,0.797
07/30/2020,AAPL,Apple Inc.,Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q3 2020 Results Confe...,"['Operator', ""Good day, everyone. Welcome to t...",/article/4362707-apple-inc-aapl-ceo-tim-cook-o...,95.851517,96.190002,96.297501,93.767502,94.187500,"158,130,000",0.482856,0.164288,1.0000,0.028,0.171,0.801
04/30/2020,AAPL,Apple Inc.,Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q2 2020 Results Confe...,"['Operator', 'Good day everyone. Welcome to th...",/article/4341792-apple-inc-aapl-ceo-tim-cook-o...,72.993935,73.449997,73.632500,72.087502,72.489998,"183,064,000",0.448222,0.193248,1.0000,0.030,0.157,0.813
01/28/2020,AAPL,Apple Inc.,Apple Inc. (AAPL) CEO Tim Cook on Q1 2020 Resu...,Apple Inc. (NASDAQ:AAPL) Q1 2020 Results Confe...,"['Operator', ""Good day, everyone. Welcome to t...",/article/4319666-apple-inc-aapl-ceo-tim-cook-o...,78.742477,79.422501,79.599998,78.047501,78.150002,"162,234,000",0.464910,0.192856,0.9999,0.024,0.165,0.811
10/30/2020,ABBV,AbbVie Inc.,AbbVie Inc.'s (ABBV) CEO Rick Gonzalez on Q3 2...,AbbVie Inc. (NYSE:ABBV) Q3 2020 Results Earnin...,"['Operator', 'Good morning and thank you for s...",/article/4383381-abbvie-inc-s-abbv-ceo-rick-go...,84.116188,85.099998,86.139999,83.000000,83.199997,"18,089,100",0.436565,0.143596,0.9999,0.017,0.133,0.850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
05/19/2020,WMT,Walmart,"Walmart, Inc. (WMT) CEO Doug McMillon on Q1 20...","Walmart, Inc. (NYSE:WMT) Q1 2021 Results Conf...",[],/article/4348814-walmart-inc-wmt-ceo-doug-mcmi...,123.985207,124.949997,131.990005,124.809998,131.750000,"24,915,400",0.000000,0.000000,0.0000,0.000,0.000,1.000
10/30/2020,XOM,Exxon Mobil Corp.,Exxon Mobil Corporation (XOM) Q3 2020 Results ...,Exxon Mobil Corporation (NYSE:XOM) Q3 2020 Ear...,"['Operator', ""Good day everyone and welcome to...",/article/4383411-exxon-mobil-corporation-xom-q...,31.851534,32.619999,33.139999,32.009998,32.410000,"47,885,700",0.382868,0.092255,0.9998,0.044,0.114,0.842
07/31/2020,XOM,Exxon Mobil Corp.,Exxon Mobil Corp (XOM) Q2 2020 Results - Earni...,Exxon Mobil Corp (NYSE:XOM) Q2 2020 Earnings C...,"['Operator', ""Good day, everyone. Welcome to t...",/article/4363013-exxon-mobil-corp-xom-q2-2020-...,40.293766,42.080002,42.169998,40.910000,41.160000,"32,123,000",0.395989,0.093709,0.9998,0.041,0.099,0.860


In [34]:
transcribed_keep2=merge_price_trans2.drop(['Name','title','transcript',
       'transcript_prep_remarks', 'url', 'Volume'],1)
transcribed_keep2

,,Adj_Close,Close,High,Low,Open,Subjectivity,Polarity,Compound,Negative,Positive,Neutral
Date,Symbol,,,,,,,,,,,
10/29/2020,AAPL,115.121384,115.320000,116.930000,112.199997,112.370003,0.492110,0.168114,1.0000,0.036,0.167,0.797
07/30/2020,AAPL,95.851517,96.190002,96.297501,93.767502,94.187500,0.482856,0.164288,1.0000,0.028,0.171,0.801
04/30/2020,AAPL,72.993935,73.449997,73.632500,72.087502,72.489998,0.448222,0.193248,1.0000,0.030,0.157,0.813
01/28/2020,AAPL,78.742477,79.422501,79.599998,78.047501,78.150002,0.464910,0.192856,0.9999,0.024,0.165,0.811
10/30/2020,ABBV,84.116188,85.099998,86.139999,83.000000,83.199997,0.436565,0.143596,0.9999,0.017,0.133,0.850
...,...,...,...,...,...,...,...,...,...,...,...,...
05/19/2020,WMT,123.985207,124.949997,131.990005,124.809998,131.750000,0.000000,0.000000,0.0000,0.000,0.000,1.000
10/30/2020,XOM,31.851534,32.619999,33.139999,32.009998,32.410000,0.382868,0.092255,0.9998,0.044,0.114,0.842
07/31/2020,XOM,40.293766,42.080002,42.169998,40.910000,41.160000,0.395989,0.093709,0.9998,0.041,0.099,0.860


In [35]:
#made label column that assigns 1 to positive values over 0.10
transcribed_keep2['Label']= np.where(transcribed_keep2['Positive']>0.100, 1, 0)

In [36]:
transcribed_keep2['Label'].shape

(401,)

In [37]:
transcribed_keep2

,,Adj_Close,Close,High,Low,Open,Subjectivity,Polarity,Compound,Negative,Positive,Neutral,Label
Date,Symbol,,,,,,,,,,,,
10/29/2020,AAPL,115.121384,115.320000,116.930000,112.199997,112.370003,0.492110,0.168114,1.0000,0.036,0.167,0.797,1
07/30/2020,AAPL,95.851517,96.190002,96.297501,93.767502,94.187500,0.482856,0.164288,1.0000,0.028,0.171,0.801,1
04/30/2020,AAPL,72.993935,73.449997,73.632500,72.087502,72.489998,0.448222,0.193248,1.0000,0.030,0.157,0.813,1
01/28/2020,AAPL,78.742477,79.422501,79.599998,78.047501,78.150002,0.464910,0.192856,0.9999,0.024,0.165,0.811,1
10/30/2020,ABBV,84.116188,85.099998,86.139999,83.000000,83.199997,0.436565,0.143596,0.9999,0.017,0.133,0.850,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
05/19/2020,WMT,123.985207,124.949997,131.990005,124.809998,131.750000,0.000000,0.000000,0.0000,0.000,0.000,1.000,0
10/30/2020,XOM,31.851534,32.619999,33.139999,32.009998,32.410000,0.382868,0.092255,0.9998,0.044,0.114,0.842,1
07/31/2020,XOM,40.293766,42.080002,42.169998,40.910000,41.160000,0.395989,0.093709,0.9998,0.041,0.099,0.860,0


In [38]:
#create featured data set
X= transcribed_keep2
X=np.array(X.drop(['Label'], 1))

#create target data set
y=np.array(transcribed_keep2['Label'])

In [39]:
X

array([[1.15121384e+02, 1.15320000e+02, 1.16930000e+02, ...,
        3.60000000e-02, 1.67000000e-01, 7.97000000e-01],
       [9.58515167e+01, 9.61900024e+01, 9.62975006e+01, ...,
        2.80000000e-02, 1.71000000e-01, 8.01000000e-01],
       [7.29939346e+01, 7.34499969e+01, 7.36324997e+01, ...,
        3.00000000e-02, 1.57000000e-01, 8.13000000e-01],
       ...,
       [4.02937660e+01, 4.20800018e+01, 4.21699982e+01, ...,
        4.10000000e-02, 9.90000000e-02, 8.60000000e-01],
       [4.05230522e+01, 4.31399994e+01, 4.63899994e+01, ...,
        4.50000000e-02, 1.33000000e-01, 8.23000000e-01],
       [5.75258255e+01, 6.21199989e+01, 6.32000008e+01, ...,
        2.90000000e-02, 1.54000000e-01, 8.17000000e-01]])

In [40]:
y

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,

In [41]:
#split the data into 80% training and 20% testing data sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=0)

In [42]:
#create and train the model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model=LinearDiscriminantAnalysis().fit(x_train, y_train)

In [43]:
#get the models predictions and classifications
predictions= model.predict(x_test)
predictions

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [44]:
#show the models metrics 
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.33      0.50         9
           1       0.92      1.00      0.96        72

    accuracy                           0.93        81
   macro avg       0.96      0.67      0.73        81
weighted avg       0.93      0.93      0.91        81

